In [12]:
import os, sys
sys.path.append(os.path.dirname(os.getcwd()))
from utils import *
import pandas as pd

In [13]:
import requests
from bs4 import BeautifulSoup

url = 'https://2021.aclweb.org/program/overview/'

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

soup

<!DOCTYPE html>

<!--
  Minimal Mistakes Jekyll Theme 4.16.4 by Michael Rose
  Copyright 2013-2019 Michael Rose - mademistakes.com | @mmistakes
  Free for personal and commercial use under the MIT license
  https://github.com/mmistakes/minimal-mistakes/blob/master/LICENSE
-->
<html class="no-js" lang="en">
<head>
<meta charset="utf-8"/>
<!-- begin _includes/seo.html --><title>ACL-IJCNLP 2021</title>
<meta content="Official website for  The Joint Conference of The 59th Annual Meeting of the Association for Computational Linguistics and The 11th International Joint Conference on Natural Language Processing (ACL-IJCNLP 2021)" name="description"/>
<meta content="website" property="og:type"/>
<meta content="en_US" property="og:locale"/>
<meta content="ACL-IJCNLP 2021" property="og:site_name"/>
<meta content="ACL-IJCNLP 2021" property="og:title"/>
<meta content="http://0.0.0.0:4000/program/overview/" property="og:url"/>
<meta content="@aclmeeting" name="twitter:site"/>
<meta content="ACL-IJC

In [14]:
selector = "#main > article > div > section > table > tbody > tr"
elements = soup.select(selector)
len(elements)

896

In [15]:
import re

papers = []
current_track = None
for element in elements:

    area_regex_match = re.search(r'(Session|Poster) \d..?: ([^\d%\(]*)( \d)?', element.text)
    if area_regex_match:
        track = area_regex_match.group(2).strip()
        current_track = track
        continue

    best_paper_session_match = re.search(r'Best Paper Session', element.text)
    if best_paper_session_match:
        # the papers in these sessions will be added manually
        # to add their tracks found in another source
        print('stop')
        break

    is_paper = element.find('em') and element.find('br')
    if is_paper:
        title = element.find('em').text.strip()
        papers.append({ 'id': None, 'title': title, 'area': track })
        continue

stop


In [16]:
# these papers were from the best paper session
# their track was in other

best_paper_session_papers = [
    ('Mind Your Outliers! Investigating the Negative Impact of Outliers on Active Learning for Visual Question Answering', 'Language Grounding to Vision, Robotics and Beyond'),
    ('All That’s ’Human’ Is Not Gold: Evaluating Human Evaluation of Generated Text', 'Language Generation'),
    ('Scientific Credibility of Machine Translation Research: A Meta-Evaluation of 769 Papers', 'Resources and Evaluation'),
    ('Neural Machine Translation with Monolingual Translation Memory', 'Machine Translation and Multilinguality'),
    ('Intrinsic Dimensionality Explains the Effectiveness of Language Model Fine-Tuning', 'Interpretability and Analysis of Models for NLP'),
    ('UnNatural Language Inference', 'Semantics: Sentence-level Semantics, Textual Inference and Other areas'),
    ('Including Signed Languages in Natural Language Processing', 'Theme'),
    ('Vocabulary Learning via Optimal Transport for Neural Machine Translation', 'Machine Translation and Multilinguality')
]

for title, track in best_paper_session_papers:
    papers.append({ 'id': None, 'title': title, 'area': track })

In [18]:
set(df['area'])

{'Computational Social Science and Cultural Analytics',
 'Dialog and Interactive Systems',
 'Discourse and Pragmatics',
 'Ethics in NLP',
 'Information Extraction',
 'Information Retrieval and Text Mining',
 'Interpretability and Analysis of Models for NLP',
 'Language Generation',
 'Language Grounding to Vision, Robotics and Beyond',
 'Linguistic Theories, Cognitive Modeling and Psycholinguistics',
 'Machine Learning for NLP',
 'Machine Translation and Multilinguality',
 'NLP Applications',
 'Phonology, Morphology and Word Segmentation',
 'Question Answering',
 'Resources and Evaluation',
 'Semantics: Lexical Semantics',
 'Semantics: Sentence-level Semantics, Textual Inference and Other areas',
 'Sentiment Analysis, Stylistic Analysis, and Argument Mining',
 'Speech and Multimodality',
 'Summarization',
 'Syntax: Tagging, Chunking, and Parsing',
 'Theme'}

In [19]:
df['source'] = 'ACL'
df['year'] = 2021
df['doi'] = None
df['abstract'] = None
df

,id,title,area,source,year,doi,abstract
0,None,Investigating label suggestions for opinion mi...,Computational Social Science and Cultural Anal...,ACL,2021,None,None
1,None,How Did This Get Funded?! Automatically Identi...,Computational Social Science and Cultural Anal...,ACL,2021,None,None
2,None,Engage the Public: Poll Question Generation fo...,Computational Social Science and Cultural Anal...,ACL,2021,None,None
3,None,HateCheck: Functional Tests for Hate Speech De...,Computational Social Science and Cultural Anal...,ACL,2021,None,None
4,None,Unified Dual-view Cognitive Model for Interpre...,Computational Social Science and Cultural Anal...,ACL,2021,None,None
...,...,...,...,...,...,...,...
737,None,Neural Machine Translation with Monolingual Tr...,Machine Translation and Multilinguality,ACL,2021,None,None
738,None,Intrinsic Dimensionality Explains the Effectiv...,Interpretability and Analysis of Models for NLP,ACL,2021,None,None
739,None,UnNatural Language Inference,"Semantics: Sentence-level Semantics, Textual I...",ACL,2021,None,None
740,None,Including Signed Languages in Natural Language...,Theme,ACL,2021,None,None


# Saving the results

In [2]:
previous_df = pd.read_csv('../data/cl_papers.csv', index_col=0)
previous_df

,id,title,area,source,year,doi,abstract
0,main.1004,AnswerFact: Fact Checking in Product Question ...,Question Answering,EMNLP,2020,10.18653/v1/2020.emnlp-main.1004,Product-related question answering platforms n...
1,main.1006,Knowledge-Grounded Dialogue Generation with Pr...,Dialog and Interactive Systems,EMNLP,2020,10.18653/v1/2020.emnlp-main.1006,We study knowledge-grounded dialogue generatio...
2,main.1009,BiST: Bi-directional Spatio-Temporal Reasoning...,Dialog and Interactive Systems,EMNLP,2020,10.18653/v1/2020.emnlp-main.1009,Video-grounded dialogues are very challenging ...
3,main.1010,A Knowledge-Aware Sequence-to-Tree Network for...,NLP Applications,EMNLP,2020,10.18653/v1/2020.emnlp-main.1010,With the advancements in natural language proc...
4,main.1011,Knowledge Association with Hyperbolic Knowledg...,Information Extraction,EMNLP,2020,10.18653/v1/2020.emnlp-main.1011,Capturing associations for knowledge graphs (K...
...,...,...,...,...,...,...,...
2432,NaN,On the Relation between Syntactic Divergence a...,"Syntax: Tagging, Chunking, and Parsing",EMNLP,2021,NaN,NaN
2433,NaN,Revisiting Tri-training of Dependency Parsers,"Syntax: Tagging, Chunking, and Parsing",EMNLP,2021,NaN,NaN
2434,NaN,Improved Latent Tree Induction with Distant Su...,"Syntax: Tagging, Chunking, and Parsing",EMNLP,2021,NaN,NaN
2435,NaN,Word Reordering for Zero-shot Cross-lingual St...,"Syntax: Tagging, Chunking, and Parsing",EMNLP,2021,NaN,NaN


In [3]:
# we'll delete previous rows from the conference
previous_df = previous_df[~((previous_df['source'] == 'ACL') & (previous_df['year'] == 2021))]
previous_df

,id,title,area,source,year,doi,abstract
0,main.1004,AnswerFact: Fact Checking in Product Question ...,Question Answering,EMNLP,2020,10.18653/v1/2020.emnlp-main.1004,Product-related question answering platforms n...
1,main.1006,Knowledge-Grounded Dialogue Generation with Pr...,Dialog and Interactive Systems,EMNLP,2020,10.18653/v1/2020.emnlp-main.1006,We study knowledge-grounded dialogue generatio...
2,main.1009,BiST: Bi-directional Spatio-Temporal Reasoning...,Dialog and Interactive Systems,EMNLP,2020,10.18653/v1/2020.emnlp-main.1009,Video-grounded dialogues are very challenging ...
3,main.1010,A Knowledge-Aware Sequence-to-Tree Network for...,NLP Applications,EMNLP,2020,10.18653/v1/2020.emnlp-main.1010,With the advancements in natural language proc...
4,main.1011,Knowledge Association with Hyperbolic Knowledg...,Information Extraction,EMNLP,2020,10.18653/v1/2020.emnlp-main.1011,Capturing associations for knowledge graphs (K...
...,...,...,...,...,...,...,...
2432,NaN,On the Relation between Syntactic Divergence a...,"Syntax: Tagging, Chunking, and Parsing",EMNLP,2021,NaN,NaN
2433,NaN,Revisiting Tri-training of Dependency Parsers,"Syntax: Tagging, Chunking, and Parsing",EMNLP,2021,NaN,NaN
2434,NaN,Improved Latent Tree Induction with Distant Su...,"Syntax: Tagging, Chunking, and Parsing",EMNLP,2021,NaN,NaN
2435,NaN,Word Reordering for Zero-shot Cross-lingual St...,"Syntax: Tagging, Chunking, and Parsing",EMNLP,2021,NaN,NaN


In [20]:
updated_df = pd.concat([previous_df, df], ignore_index=True)
updated_df

,id,title,area,source,year,doi,abstract
0,main.1004,AnswerFact: Fact Checking in Product Question ...,Question Answering,EMNLP,2020,10.18653/v1/2020.emnlp-main.1004,Product-related question answering platforms n...
1,main.1006,Knowledge-Grounded Dialogue Generation with Pr...,Dialog and Interactive Systems,EMNLP,2020,10.18653/v1/2020.emnlp-main.1006,We study knowledge-grounded dialogue generatio...
2,main.1009,BiST: Bi-directional Spatio-Temporal Reasoning...,Dialog and Interactive Systems,EMNLP,2020,10.18653/v1/2020.emnlp-main.1009,Video-grounded dialogues are very challenging ...
3,main.1010,A Knowledge-Aware Sequence-to-Tree Network for...,NLP Applications,EMNLP,2020,10.18653/v1/2020.emnlp-main.1010,With the advancements in natural language proc...
4,main.1011,Knowledge Association with Hyperbolic Knowledg...,Information Extraction,EMNLP,2020,10.18653/v1/2020.emnlp-main.1011,Capturing associations for knowledge graphs (K...
...,...,...,...,...,...,...,...
3174,None,Neural Machine Translation with Monolingual Tr...,Machine Translation and Multilinguality,ACL,2021,None,None
3175,None,Intrinsic Dimensionality Explains the Effectiv...,Interpretability and Analysis of Models for NLP,ACL,2021,None,None
3176,None,UnNatural Language Inference,"Semantics: Sentence-level Semantics, Textual I...",ACL,2021,None,None
3177,None,Including Signed Languages in Natural Language...,Theme,ACL,2021,None,None


In [21]:
updated_df.to_csv('../data/cl_papers.csv')